## The extended IMDB 5000 Movie Database analysis


In [1]:
import pandas as pd  
import re  
from string import printable  
import time

Read the data from a .CSV fie and check the types

In [2]:
df = pd.read_csv("data/movie_metadata_upd.csv",parse_dates=[-2],infer_datetime_format=True)
df.head()

,Unnamed: 0,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,...,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,rel_date,tagline
0,0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,...,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000,2009-12-17,Enter the World
1,1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,...,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0,2007-05-24,"At the End of the World, the Adventure Begins"
2,2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,...,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000,2015-11-12,The Dead Are Alive
3,3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,...,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000,2012-07-19,Rise
4,5,Color,Andrew Stanton,462.0,132.0,475.0,530.0,Samantha Morton,640.0,73058679.0,...,USA,PG-13,263700000.0,2012.0,632.0,6.6,2.35,24000,2012-03-08,Lost in Our World. Found in Another.


Let's see what types are there:

In [3]:
df.dtypes

Unnamed: 0                            int64
color                                object
director_name                        object
num_critic_for_reviews              float64
duration                            float64
director_facebook_likes             float64
actor_3_facebook_likes              float64
actor_2_name                         object
actor_1_facebook_likes              float64
gross                               float64
genres                               object
actor_1_name                         object
movie_title                          object
num_voted_users                       int64
cast_total_facebook_likes             int64
actor_3_name                         object
facenumber_in_poster                float64
plot_keywords                        object
movie_imdb_link                      object
num_user_for_reviews                float64
language                             object
country                              object
content_rating                  

It's rather important to know on which day of week a movie was released. Also, what quarter? We add the corresponding columns for conveninece.

In [4]:
df["rel_weekday"] = df["rel_date"].dt.weekday  # Monday=0, Sunday=6
df["rel_quarter"] = df["rel_date"].dt.quarter  # Jan=Mar = 1, Apr-Jun = 2, etc.

In [5]:
df.head()

,Unnamed: 0,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,...,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,rel_date,tagline,rel_weekday,rel_quarter
0,0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,...,237000000.0,2009.0,936.0,7.9,1.78,33000,2009-12-17,Enter the World,3.0,4.0
1,1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,...,300000000.0,2007.0,5000.0,7.1,2.35,0,2007-05-24,"At the End of the World, the Adventure Begins",3.0,2.0
2,2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,...,245000000.0,2015.0,393.0,6.8,2.35,85000,2015-11-12,The Dead Are Alive,3.0,4.0
3,3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,...,250000000.0,2012.0,23000.0,8.5,2.35,164000,2012-07-19,Rise,3.0,3.0
4,5,Color,Andrew Stanton,462.0,132.0,475.0,530.0,Samantha Morton,640.0,73058679.0,...,263700000.0,2012.0,632.0,6.6,2.35,24000,2012-03-08,Lost in Our World. Found in Another.,3.0,1.0


Now load the name gazetteers

In [32]:
male_names = []
female_names = []

with open("data/person_female.txt","r") as f:
    for line in f:
        female_names.append(line.strip())
print("got a list of {} female names".format(len(female_names)))

with open("data/person_male.txt","r") as f:
    for line in f:
        male_names.append(line.strip())
print("got a list of {} male names".format(len(male_names)))

got a list of 5732 female names
got a list of 4363 male names


In [37]:
"""
create a list of M/F for a series of names
"""
def getSexList(ser):
    li = ["None"]*len(ser)
    _f_name = ser.astype(str).apply(lambda _: _.strip().split()[0])
    #print(_f_name)
    _male_idx = _f_name.isin(male_names)
    #print(_male_idx)
    _female_idx = _f_name.isin(female_names)
    for j in range(len(li)):
        #print(_male_idx[j])
        if _male_idx[j]:
            li[j] = "M"
        else:
            li[j] = "F"    
    return li

df["director_sex"] = getSexList(df["director_name"])
df["actor_1_sex"] = getSexList(df["actor_1_name"])
df["actor_2_sex"] = getSexList(df["actor_2_name"])
df["actor_3_sex"] = getSexList(df["actor_3_name"])

In [38]:
df["director_sex"].count()

4353

In [39]:
df["actor_1_sex"].count()

4353

In [40]:
df.iloc[:,29:]

,rel_date,tagline,rel_weekday,rel_quarter,director_sex,actor_1_sex,actor_2_sex,actor_3_sex
0,2009-12-17,Enter the World,3.0,4.0,M,F,M,M
1,2007-05-24,"At the End of the World, the Adventure Begins",3.0,2.0,F,M,F,M
2,2015-11-12,The Dead Are Alive,3.0,4.0,M,M,M,F
3,2012-07-19,Rise,3.0,3.0,M,M,F,M
4,2012-03-08,Lost in Our World. Found in Another.,3.0,1.0,M,M,F,F
5,2007-05-03,The battle within.,3.0,2.0,M,F,M,F
6,2011-01-06,Get tangled up.,3.0,1.0,M,M,F,F
7,2015-04-23,A new age begins,3.0,2.0,F,M,M,F
8,2009-07-15,Dark Secrets Revealed,2.0,3.0,M,M,M,M
9,2016-03-24,Who will win?,3.0,1.0,M,M,F,M
